# Mall Customer Segmentation
    - goal is to know which one is our best customer profile & focus attract those people AND find out why are they interested to us

# Obtain

# Scrub

# Expore

# Model

# iNterpret